N-dimensional functions:

collision calculates the place where a particle with position x1 and velocity v collide with an obstacle of radius r and centre x2. If there is not collision returns "false".

velo_col, calculates de velocity after the collision, if the collision takes place at position x1, with an obstacle with centre x2, and initial velocity v.


In [16]:
function collision(x1,x2,r,v)
    v2=dot(v,v)
    b=dot((x1-x2),v)/v2
    c=(dot((x1-x2),(x1-x2))-r^2)/v2
    if(BigFloat(b^2-c)<BigFloat("0"))
        return false, 0
    end
    t=-b-sqrt(b^2-c)
    x=v*t+x1
    return x, t

end

function velo_col(x1,x2,v)
    n=(x1-x2)
    n=n/norm(n)
    vn=dot(n,v)*n
    v=v-2*vn
    v=v/norm(v)
    return v
end

velo_col (generic function with 1 method)

2D effitient algorithm:

function $frac(\alpha,\epsilon)$ return the smallest integrate numbers, $p$ and $q$, such that $|\alpha−p/q|<\epsilon/q$. It use the continued fraction develpement.

nextt(x,v,r) calculates the first of the next possibilities: The interesection of a particle at a initial position $x$ and velocity $v$, with the next line of the forme $(n,y)$, where $n$ is an integrer and $y$ is the variable or the intersection of the particle an one of the obstacles of radious $r$, and center at $int(x)$, or $int(x)+e1$, or $int(x)+e2$, or $int(x)+e1+e2$.

eff(m,b,r) calculates the center of the next obstacle of radious r with which a particle will collide in a squere arrangement, if the particle have a position $x=(0,b)$ and a velocity $(vx,vy)$ such that $0<vy/vx<1$ and $vy>0$.

Lor(x,v,r) calculates using next and eff, the center of the next obstacle of radious r with which a particle with initial position x and velocity v=[vx,vy] will collide in a squere arrangement if vx>vy>0

Finally, Lorentz(x,v,r) use rotations and reflexions to generalize Lor(x,v,r) to any velocity

In [17]:
function frac(x, epsilon)
   h1, h2 = 1, 0
   k1, k2 = 0, 1
   b = x
   while abs(k1*x - h1) > epsilon
       a = ifloor(b)
       h1, h2 = a*h1 + h2, h1
       k1, k2 = a*k1 + k2, k1
       b = 1/(b - a)
   end
   return k1, h1
end



function nextt1(x,v,r) 
    e1=[1,0]
    e2=[0,1]
    nn=int(x)
    x1=x-nn      
    t=(-x1[1])/v[1]
    test=1
    if(t<0)
        t=(1-x1[1])/v[1] 
        test=2
    end
    xx1=x1+v*t
    b=xx1[2]    
    epsilon=r/v[1]
    testt=0
    if(test==1)
        if((b>-epsilon && b<epsilon))
            return nn, testt
        elseif((1-b>-epsilon) && (1-b<epsilon))
            return nn+e2, testt
        end       
    elseif(test==2)
        if((b>-epsilon && b<epsilon))
            return nn+e1, testt
        elseif(1-b>-epsilon && 1-b<epsilon) 
            return nn+e1+e2,testt
        end
    end
    testt=1
    return nn+xx1, testt
end

function eff(m, b, r)
	kn = 0
    b1=b
    epsilon=r*sqrt(m*m+1) 
    if(b < epsilon || 1 - b < epsilon)
        if b < BigFloat("0.5")
			(q, p) = frac(m, 2.*b)
		else
			(q, p) = frac(m, 2.*(1. - b))
		end
		b = mod(m*q + b, 1)
		kn += q
    end  
	while b > epsilon && 1 - b > epsilon
		if b < BigFloat("0.5")
			(q, p) = frac(m, 2.*b)
		else
			(q, p) = frac(m, 2.*(1. - b))
		end
		b = mod(m*q + b, 1)
		kn += q
	end
	q = kn
    p = int(m*q+b1)
    return [q, p]
end

function Lor(x,v,r)   #This function obtain the coordenates of the obstacle with which 
                    #a particle at a initial position x, and intial velocity v collids 
                    #if both components of the velocity are positive, and the slope is less than 1.
    x1,test=nextt1(x,v,r)
    if(test==0)
        return x1
    end
    v1=v[1]
    v2=v[2]
    m=v2/v1
    b=x1[2]
    b=b-floor(b)
    de=[0, int(b)]
    centro=eff(m,b,r)
    x2=int(x1)-de+centro
    return x2
end

function Lorentz1(x,v,r)  
    ROT=[[0 1],
        [-1 0]]  #Rotational matrix π/2 radians
    REF=[[0 1],
        [1 0]]   #Reflect matrix, change y->x and x->y
    ROT2=ROT^2
    ROT3=ROT^3
    v1=v[1]
    v2=v[2]
    vv=copy(v)
    xx=copy(x)
    m1=v2/v1
    if(norm(int(x)-x)<r)   #if a particle begin inside an obstacle, then the first collision 
                          #is considered with the same obstacle.
        return int(x)
    end
    if(m1>=0 && v2>=0)  # if the velocity is in the quadrant I 
        if(m1<=1)
            xx=Lor(xx,vv,r)
        elseif(m1>1)  
            xx=REF*xx
            vv=REF*vv
            xx=Lor(xx,vv,r)
            xx=REF*xx
            vv=REF*vv            
        end
        return xx
    elseif(m1>=0 && v2<0) #if the velocity is in the quadrant III
        xx=ROT2*xx
        vv=ROT2*vv
        if(m1<=1)
            xx=Lor(xx,vv,r)  
        elseif(m1>1)
            xx=REF*xx
            vv=REF*vv
            xx=Lor(xx,vv,r)
            xx=REF*xx
            vv=REF*vv   
        end
        xx=ROT2*xx
        vv=ROT2*vv
        return xx
    elseif(m1<0 && v2>=0) #if the velocity is in the quadrant II
        xx=ROT*xx
        vv=ROT*vv
        if(m1<-1)
            xx=Lor(xx,vv,r)            
        elseif(m1>=-1)
            xx=REF*xx
            vv=REF*vv
            xx=Lor(xx,vv,r)
            xx=REF*xx   
            vv=REF*vv 
        end     
        xx=ROT3*xx
        vv=ROT3*vv
        return xx
    elseif(m1<0 && v2<0) #if the velocity is in the quadrant IV
        xx=ROT3*xx
        vv=ROT3*vv
        if(m1<-1)
            xx=Lor(xx,vv,r)        
        elseif(m1>=-1)
            xx=REF*xx
            vv=REF*vv
            xx=Lor(xx,vv,r)
            xx=REF*xx
            vv=REF*vv 
        end   
        xx=ROT*xx
        vv=ROT3*vv
        return xx
    end
end

Lorentz1 (generic function with 1 method)

Now, to do a simulation, we can use center=Lorentz1(x,v,r) to obtain the coordenates of the obstacle, then use x=collision(x,center,r,v) to calculate the exact point of collision, and then v=velo_col(x,center,v) to calculate the new velocity. Then the whole simulation will be:

In [3]:
function LorentzGas1(x,v,r,steps)
    for i=1:steps
        center=Lorentz1(x,v,r) 
        x, t=collision(x,center,r,v)
        v=velo_col(x,center,v)
    end
    return x, v
end 

LorentzGas1 (generic function with 1 method)

In [4]:
r=BigFloat("0.00001")
x=rand(2).*BigFloat[1,1]
angle=BigFloat(rand(1)[1])*pi*2.
v=[cos(angle), sin(angle)]
v=v/norm(v)
steps=1000
x,v=LorentzGas1(x,v,r,steps)


(BigFloat[-1.657015999998518068428190698969643043655801547206328012815233679764980257587239e+06,-6.35283999990110415639852460692106090797176005113787140958621602087960273445551e+05],BigFloat[9.75640576905507223582802136865841273669565226128671068527141331268251702644458e-01,2.193751688215624936367456504400192968314327003189632458584796023079036132267473e-01])

Or, to calculate using a maximum time instead of steps to calculate diffusion:

In [5]:
function LorentzGas2(x,v,r,time)
    t=0
    v1=copy(v)
    while(t<time)
        x1=copy(x)
        v1=copy(v)
        center=Lorentz1(x,v,r) 
        x, tt=collision(x,center,r,v)
        v=velo_col(x,center,v)
        t=t+tt #norm(x-x1)
    end
    t=t-time
    if(t==0)
        v1=v
    end
    x=x-t*v1
    return x, v1
end 

LorentzGas2 (generic function with 1 method)

A 3D version of the Lorentz gas can be produced by looking on 3 of the proyected planes xy, xz, and yz. Here it is:

In [6]:
function Lorentz3D(x,v,r)
    if(norm(int(x)-x)<r)   #if a particle begin inside an obstacle, then the first collision 
                          #is considered with the same obstacle.
    xx, t=collision(x,int(x),r,v)
        return xx
    end
    x1=zeros(2)
    x2=zeros(2)
    x3=zeros(2)
    xr=false
    while (xr==false)
        v_xy=[v[1], v[2]]
        v_xz=[v[1], v[3]]
        v_yz=[v[2], v[3]]                #Velocities in the planes xy, xz, and yz
        vn_xy=v_xy./BigFloat(sqrt(dot(v_xy,v_xy)))
        vn_xz=v_xz./BigFloat(sqrt(dot(v_xz,v_xz)))
        vn_yz=v_yz./BigFloat(sqrt(dot(v_yz,v_yz))) #normalized velocities in the planes xy, xz, and yz
        tmax=BigFloat("0")
        tmin=BigFloat("-2")
        x1[1]=1
        x1[2]=2
        x2[1]=3
        x2[2]=4
        x3[1]=5 
        x3[2]=6
        #        while (int(tmax)>int(tmin+BigFloat("0.5")) )   #it can be considerin time conincidence
        while(x1[1]!=x2[1] || x1[2]!=x3[1] || x2[2]!=x3[2])    #or considering the obstacle coincidence.
            x_xy=[x[1], x[2]]
            x_xz=[x[1], x[3]]
            x_yz=[x[2], x[3]]                #positions in the planes xy, xz, and yz

            x1=Lorentz1(x_xy,vn_xy,r)#calculate the position of obstacle that collide with the particle 
            x2=Lorentz1(x_xz,vn_xz,r)# in the 2D Lorentz gas for the 3 planes
            x3=Lorentz1(x_yz,vn_yz,r) 
            t1=(sqrt(dot(x1-x_xy,x1-x_xy))-r)/sqrt(dot(v_xy,v_xy))
            t2=(sqrt(dot(x2-x_xz,x2-x_xz))-r)/sqrt(dot(v_xz,v_xz))
            t3=(sqrt(dot(x3-x_yz,x3-x_yz))-r)/sqrt(dot(v_yz,v_yz))  #calculate the time used to reach the obstacle. 

            tmax=maximum([t1 t2 t3])    # if the minumum and maximum valueds of time are almost the same, then 
            tmin=minimum([t1 t2 t3])    # there is very high probable a collision  
            x=x+v*(tmax-0.1*r)   

        end
        xr, tt=collision(x,int(x),r,v) #calculates the collision. In case that there is not, then, it advance the particle to the 
                                   #next cell
        if(tt<0)
            xr=false
        end
        x=x+v*(1-2*r)
    end
    return xr
end

Lorentz3D (generic function with 1 method)

In [7]:
function ClassicLorentz(x,v,r)    #This is a simpler and inefficient version of the Classic algorithm for ND. Just to 
                                  #check if the 3D version works correctly.
   if(norm(int(x)-x)<r)   #if a particle begin inside an obstacle, then the first collision 
                          #is considered with the same obstacle.
        xx,tt=collision(x,int(x),r,v)
        return xx
    end
    xr=false
    i=0.0
    while (xr==false)
        while (xr==false)
            xr, tt=collision(x,int(x+v*(i)),r,v)
            i=i+1-2*r
        end
        t=sqrt(dot(xr-x,xr-x)/dot(v,v))
        if(dot(x+t*v-xr,x+t*v-xr)>dot(xr-x,xr-x))
            xr=false
        end
    end
    return xr
end

ClassicLorentz (generic function with 1 method)

In [44]:
for j=1:5
    time1=0
    time2=0
    r=0.4^j
    for i=1:100
        x=rand(3).*BigFloat[1,1,1]
        v=rand(3).*BigFloat[1,1,1] #-BigFloat[0.5,0.5,0.5]
        v=v/norm(v)
        tic()
        x1=Lorentz3D2(x,v,r)
        t1=toq()
        tic()
        x2=Lorentz3D(x,v,r)
        t2=toq()
        time1=time1+t1
        time2=time2+t2
        if(sum(int(x1-x2))>0)
            println(int(x1), int(x2), int(x*100)/100,int(v*100)/100,norm(int(x)-x) )
        end
    end
    println("time1=", time1, "  time2=", time2, " ratio=", time2/time1)    
end
0.4^6

time1=0.067178835  time2=0.06697243100000001 ratio=0.996927544218354
time1=0.17672230700000002  time2=0.173809512 ratio=0.9835176721634806
time1=1.4460555919999998  time2=0.9362394459999996 ratio=0.6474436053354716
time1=5.092122469000001  time2=4.778966806 ratio=0.9385019380609085
time1=8.44920818  time2=7.669998437000004 ratio=0.9077771873529579


0.0040960000000000015

In [43]:
r=BigFloat("0.02")
v = [3, 4, 5].*BigFloat[1,1,1]
    v=v/norm(v)
x=[0.555774, 0.401411, 0.293452].*BigFloat[1,1,1]
    x2=Lorentz3D2(x,v,r)
0.4^10

0.00010485760000000006

In [19]:
BigFloat(9.666494000000000252903475939092459157109260559082031250000000000000000000000138-9.666494000000000252903475939092459157109260559082031250000000000000000000000052)

0e+00 with 256 bits of precision

In [25]:
function eff2(m, b, r)
	kn = 0
    b1=b
    epsilon=r*sqrt(m*m+1) 
    if(b < epsilon || 1 - b < epsilon)
        if b < BigFloat("0.5")
			(q, p) = frac(m, 2.*b)
		else
			(q, p) = frac(m, 2.*(1. - b))
		end
		b = mod(m*q + b, 1)
		kn += q
    end  
    b1=copy(b)
	while b > epsilon && 1 - b > epsilon
		if b < BigFloat("0.5")
			(q, p) = frac(m, 2.*b)
		else
			(q, p) = frac(m, 2.*(1. - b))
		end
		b = mod(m*q + b, 1)
		kn += q
#        println(b, "  ", b1)
        if(abs(b-b1)<BigFloat(0.000000000000000000000000000000000000000000000000001) )
            return false
        end
	end
	q = kn
    p = int(m*q+b1)
    return [q, p]
end

eff2 (generic function with 1 method)

In [26]:

function Lor2(x,v,r)   #This function obtain the coordenates of the obstacle with which 
                    #a particle at a initial position x, and intial velocity v collids 
                    #if both components of the velocity are positive, and the slope is less than 1.
    x1,test=nextt1(x,v,r)
    if(test==0)
        return x1
    end
    v1=v[1]
    v2=v[2]
    m=v2/v1
    b=x1[2]
    b=b-floor(b)
    de=[0, int(b)]
    centro=eff2(m,b,r)
    if(centro==false)
        return false
    end
    x2=int(x1)-de+centro
    return x2
end

function Lorentz2(x,v,r)  
    ROT=[[0 1],
        [-1 0]]  #Rotational matrix π/2 radians
    REF=[[0 1],
        [1 0]]   #Reflect matrix, change y->x and x->y
    ROT2=ROT^2
    ROT3=ROT^3
    v1=v[1]
    v2=v[2]
    vv=copy(v)
    xx=copy(x)
    m1=v2/v1
    if(norm(int(x)-x)<r)   #if a particle begin inside an obstacle, then the first collision 
                          #is considered with the same obstacle.
        return int(x)
    end
    if(m1>=0 && v2>=0)  # if the velocity is in the quadrant I 
        if(m1<=1)
            xx=Lor2(xx,vv,r)
            if(xx==false)
                return false
            end
        elseif(m1>1)  
            xx=REF*xx
            vv=REF*vv
            xx=Lor2(xx,vv,r)
            if(xx==false)
                return false
            end
            xx=REF*xx
            vv=REF*vv            
        end
        return xx
    elseif(m1>=0 && v2<0) #if the velocity is in the quadrant III
        xx=ROT2*xx
        vv=ROT2*vv
        if(m1<=1)
            xx=Lor2(xx,vv,r)  
            if(xx==false)
                return false
            end
        elseif(m1>1)
            xx=REF*xx
            vv=REF*vv
            xx=Lor2(xx,vv,r)
            if(xx==false)
                return false
            end
            xx=REF*xx
            vv=REF*vv   
        end
        xx=ROT2*xx
        vv=ROT2*vv
        return xx
    elseif(m1<0 && v2>=0) #if the velocity is in the quadrant II
        xx=ROT*xx
        vv=ROT*vv
        if(m1<-1)
            xx=Lor2(xx,vv,r) 
            if(xx==false)
                return false
            end            
        elseif(m1>=-1)
            xx=REF*xx
            vv=REF*vv
            xx=Lor2(xx,vv,r)
            if(xx==false)
                return false
            end            
            xx=REF*xx   
            vv=REF*vv 
        end     
        xx=ROT3*xx
        vv=ROT3*vv
        return xx
    elseif(m1<0 && v2<0) #if the velocity is in the quadrant IV
        xx=ROT3*xx
        vv=ROT3*vv
        if(m1<-1)
            xx=Lor2(xx,vv,r)  
            if(xx==false)
                return false
            end                
        elseif(m1>=-1)
            xx=REF*xx
            vv=REF*vv
            xx=Lor2(xx,vv,r)
            if(xx==false)
                return false
            end          
            xx=REF*xx
            vv=REF*vv 
        end   
        xx=ROT*xx
        vv=ROT3*vv
        return xx
    end
end

Lorentz2 (generic function with 1 method)

In [32]:
function Lorentz3D2(x,v,r)
    if(norm(int(x)-x)<r)   #if a particle begin inside an obstacle, then the first collision 
                          #is considered with the same obstacle.
    xx, t=collision(x,int(x),r,v)
        return xx
    end
    x1=zeros(2)
    x2=zeros(2)
    x3=zeros(2)
    xr=false
    while (xr==false)
        v_xy=[v[1], v[2]]
        v_xz=[v[1], v[3]]
        v_yz=[v[2], v[3]]                #Velocities in the planes xy, xz, and yz
        vn_xy=v_xy./BigFloat(sqrt(dot(v_xy,v_xy)))
        vn_xz=v_xz./BigFloat(sqrt(dot(v_xz,v_xz)))
        vn_yz=v_yz./BigFloat(sqrt(dot(v_yz,v_yz))) #normalized velocities in the planes xy, xz, and yz
        tmax=BigFloat("0")
        tmin=BigFloat("-2")
        x1[1]=1
        x1[2]=2
        x2[1]=3
        x2[2]=4
        x3[1]=5 
        x3[2]=6
        #        while (int(tmax)>int(tmin+BigFloat("0.5")) )   #it can be considerin time conincidence
        while(x1[1]!=x2[1] || x1[2]!=x3[1] || x2[2]!=x3[2])    #or considering the obstacle coincidence.
            x_xy=[x[1], x[2]]
            x_xz=[x[1], x[3]]
            x_yz=[x[2], x[3]]                #positions in the planes xy, xz, and yz

            x1=Lorentz2(x_xy,vn_xy,r)#calculate the position of obstacle that collide with the particle 
            if(x1==false)
                return false
            end
            x2=Lorentz2(x_xz,vn_xz,r)# in the 2D Lorentz gas for the 3 planes
            if(x2==false)
                return false
            end
            x3=Lorentz2(x_yz,vn_yz,r) 
            if(x3==false)
                return false
            end
            t1=(sqrt(dot(x1-x_xy,x1-x_xy))-r)/sqrt(dot(v_xy,v_xy))
            t2=(sqrt(dot(x2-x_xz,x2-x_xz))-r)/sqrt(dot(v_xz,v_xz))
            t3=(sqrt(dot(x3-x_yz,x3-x_yz))-r)/sqrt(dot(v_yz,v_yz))  #calculate the time used to reach the obstacle. 

            tmax=maximum([t1 t2 t3])    # if the minumum and maximum valueds of time are almost the same, then 
            tmin=minimum([t1 t2 t3])    # there is very high probable a collision  
            x=x+v*(tmax-0.1*r)   

        end
        xr, tt=collision(x,int(x),r,v) #calculates the collision. In case that there is not, then, it advance the particle to the 
                                   #next cell
        if(tt<0)
            xr=false
        end
        x=x+v*(1-2*r)
    end
    return xr
end

Lorentz3D2 (generic function with 1 method)